In [2]:
# import os
# import sys

# sys.dont_write_bytecode = True
# os.environ["NUMBA_DISABLE_JIT"] = "1"


In [3]:
import numpy as np
import pandas as pd
from my_stuff import MufexKeys
from nb_quantfreedom.exchanges.mufex_exchange.mufex import Mufex
from nb_quantfreedom.nb_enums import DynamicOrderSettingsArrays
from nb_quantfreedom.utils import pretty_qf
from requests import get
from talib import RSI
from numba import njit
np.set_printoptions(formatter={"float_kind": "{:.2f}".format})

pd.options.display.float_format = "{:.2f}".format

mufex_main = Mufex(
    api_key=MufexKeys.api_key,
    secret_key=MufexKeys.secret_key,
    use_test_net=False,
)

%load_ext autoreload
%autoreload 2


In [18]:
candles = mufex_main.get_candles_df(
    symbol="BTCUSDT",
    timeframe="15m",
    candles_to_dl=200,
)

It took 00 mins and 00 seconds to download 200 candles


In [32]:
from nb_quantfreedom.nb_enums import CandleBodyType, RejectedOrder


@njit()
def try_func():
    msg = (
        "klsjdflkjsdf "
        + str(4545345)
        + "\naskdjfwoierwejk "
        + str(4895908273.5651651)
        + "aksjdfkljasdkfjasdkjf alskjdf lkajsdf ;lkjsdfl;kjsdf ;lkjsaf laksfj "
        + str(349857349857) + None
    )
    print(msg)


try_func()

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1mNo implementation of function Function(<built-in function add>) found for signature:
 
 >>> add(unicode_type, none)
 
There are 20 candidate implementations:
[1m  - Of which 18 did not match due to:
  Overload of function 'add': File: <numerous>: Line N/A.
    With argument(s): '(unicode_type, none)':[0m
[1m   No match.[0m
[1m  - Of which 2 did not match due to:
  Operator Overload in function 'add': File: unknown: Line unknown.
    With argument(s): '(unicode_type, none)':[0m
[1m   No match for registered cases:
    * (int64, int64) -> int64
    * (int64, uint64) -> int64
    * (uint64, int64) -> int64
    * (uint64, uint64) -> uint64
    * (float32, float32) -> float32
    * (float64, float64) -> float64
    * (complex64, complex64) -> complex64
    * (complex128, complex128) -> complex128[0m
[0m
[0m[1mDuring: typing of intrinsic-call at C:\Users\User\AppData\Local\Temp\ipykernel_21040\13564457

In [ ]:
from nb_quantfreedom.nb_helper_funcs import nb_float_to_str, pass_float_to_str, reg_float_to_str


@njit()
def try_func(tester: callable):
    return tester(513.165)



try_func(nb_float_to_str)

In [ ]:
candles = mufex_main.get_candles_df(
    symbol="BTCUSDT",
    timeframe="30m",
    limit=200
)
pd.DataFrame(candles)

In [ ]:
"hello " + str(3949)

In [ ]:
issubclass(StopIteration,Exception)

In [ ]:
Exception.__subclasses__()

In [ ]:
for i in range(5000):
    closing_p = candles[:, 4]
    up = np.zeros(closing_p.size)
    down = np.zeros(closing_p.size)
    mean_up = np.zeros(closing_p.size)
    mean_down = np.zeros(closing_p.size)
    rsi = np.full(closing_p.size, np.nan)
    for counter in range(1, 13):
        pchg = (closing_p[counter] - closing_p[counter - 1]) / closing_p[counter - 1]
        if pchg > 0:
            up[counter] = pchg
        else:
            down[counter] = abs(pchg)

    for counter in range(14, 200):
        pchg = (closing_p[counter] - closing_p[counter - 1]) / closing_p[counter - 1]
        if pchg > 0:
            up[counter] = pchg
        else:
            down[counter] = abs(pchg)
        mean_up[counter] = np.mean(up[counter - 14 : counter + 1])
        mean_down[counter] = np.mean(down[counter - 14 : counter + 1])
        rsi[counter] = 100 - 100 / (1 + (mean_up[counter] / mean_down[counter]))

In [ ]:
@njit(cache=True)
def numba_test():
    closing_p = candles[:, 4]
    up = np.zeros(closing_p.size)
    down = np.zeros(closing_p.size)
    mean_up = np.zeros(closing_p.size)
    mean_down = np.zeros(closing_p.size)
    rsi = np.full(closing_p.size, np.nan)
    for counter in range(1, 13):
        pchg = (closing_p[counter] - closing_p[counter - 1]) / closing_p[counter - 1]
        if pchg > 0:
            up[counter] = pchg
        else:
            down[counter] = abs(pchg)

    for counter in range(14, 200):
        pchg = (closing_p[counter] - closing_p[counter - 1]) / closing_p[counter - 1]
        if pchg > 0:
            up[counter] = pchg
        else:
            down[counter] = abs(pchg)
        mean_up[counter] = np.mean(up[counter - 14 : counter + 1])
        mean_down[counter] = np.mean(down[counter - 14 : counter + 1])
        rsi[counter] = 100 - 100 / (1 + (mean_up[counter] / mean_down[counter]))
for i in range(500000):
    numba_test()

In [ ]:
from nb_quantfreedom.indicators.indicators import IndicatorsNB
from nb_quantfreedom.nb_enums import CandleBodyType


@njit(cache=True)
def tester():
    return IndicatorsNB().calc_rsi(prices=candles[:, CandleBodyType.Close], period=14)

for i in range(500000):
    tester()